In [1]:
import openai

import yaml
with open("conf/index.yaml") as f:
    credentials = yaml.safe_load(f)
openai.api_key = credentials['environment_variables']['OPENAI_API_KEY']
model_name = "gpt-4"
system_prompt = ""
user_query = ""

In [2]:
import pandas as pd

df = pd.read_csv('./data/ground_truth.csv')
print(df.head(10))
df = df.head(100)
arguments = list(df['argument'].values)
target_KPs = list(df['key_point'].values)
print(arguments)
print(len(arguments))

                                           topic  \
0  Assisted suicide should be a criminal offence   
1  Assisted suicide should be a criminal offence   
2  Assisted suicide should be a criminal offence   
3  Assisted suicide should be a criminal offence   
4  Assisted suicide should be a criminal offence   
5  Assisted suicide should be a criminal offence   
6  Assisted suicide should be a criminal offence   
7  Assisted suicide should be a criminal offence   
8  Assisted suicide should be a criminal offence   
9  Assisted suicide should be a criminal offence   

                                            argument  \
0  a cure or treatment may be discovered shortly ...   
1  A patient should be able to decide when they h...   
2  a person has the right to end their suffering ...   
3  a person should have the dignity to choose how...   
4  a person should have the right to be able to c...   
5  a person should have the right to die on their...   
6  a person should not be forced in

### Few Shot

In [3]:
predict_KPs = []
system_prompt = \
        """
        You need to do key point analysis on the query of user and
        generate a key point from it. One example is:
        Argument: a cure or treatment may be discovered shortly after having ended someone's life
        unnecessarily.
        Key Point: Assisted suicide should not be allowed because many times people can still get better
        """
for argument in arguments:
    rep = openai.ChatCompletion.create(
        model = model_name,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": argument}
        ]
    )

    result = rep["choices"][0]["message"]["content"]
    print(len(predict_KPs))
    print(result)
    predict_KPs.append(result)

0
Key Point: Euthanasia or assisted suicide could prematurely end lives before potential treatments or cures are discovered.
1
Key Point: Patient autonomy is crucial in decisions regarding end-of-life care.
2
Key Point: Assisted suicide should be accepted as it gives individuals the right to end their own suffering and those who assist them shouldn't be penalized.
3
Key Point: Assisted suicide should be allowed as it empowers individuals to decide their end-of-life choices.
4
Key Point: People should have the autonomy to make decisions about their own life and death.
5
Key Point: The legality of assisted suicide respects individual autonomy and personal choice in ending one's own life.
6
Key Point: Assisted suicide should be allowed to prevent prolonged suffering when death is inevitable.
7
Key Point: Assisted suicide can be misused for financial motives.
8
Key Point: Assisted suicide is equated with murder, regardless of the person's consent.
9
Key Point: Assisted suicide is questiona

In [4]:
# print(result)
# print(result[11:])
# print(predict_KPs)
for i in range(len(predict_KPs)):
    predict_KPs[i] = predict_KPs[i][11:]
# print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(predict_KPs)

['Euthanasia or assisted suicide could prematurely end lives before potential treatments or cures are discovered.', 'Patient autonomy is crucial in decisions regarding end-of-life care.', "Assisted suicide should be accepted as it gives individuals the right to end their own suffering and those who assist them shouldn't be penalized.", 'Assisted suicide should be allowed as it empowers individuals to decide their end-of-life choices.', 'People should have the autonomy to make decisions about their own life and death.', "The legality of assisted suicide respects individual autonomy and personal choice in ending one's own life.", 'Assisted suicide should be allowed to prevent prolonged suffering when death is inevitable.', 'Assisted suicide can be misused for financial motives.', "Assisted suicide is equated with murder, regardless of the person's consent.", 'Assisted suicide is questionable because there are often cases where people recover through alternative treatments or medical adva

### BLEU Score

In [11]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import nltk
nltk.download('punkt')

# Tokenize sentences into lists of words
tokenized_candidates = [nltk.word_tokenize(sentence) for sentence in predict_KPs]
tokenized_references = [[nltk.word_tokenize(reference)] for reference in target_KPs]

# Calculate BLEU score for the entire corpus
bleu_score = corpus_bleu(tokenized_references, tokenized_candidates)

print("BLEU Score:", bleu_score)

BLEU Score: 0.02409877058871543


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jhluo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
import sacrebleu

# Example sentences
candidates = predict_KPs
references = target_KPs

# Join sentences into strings
candidate_str = "\n".join(candidates)
reference_str = "\n".join(references)

# Calculate BLEU score
bleu = sacrebleu.raw_corpus_bleu(candidate_str, [reference_str])
print("BLEU Score:", bleu.score)

BLEU Score: 5.68609022556391


### BERT Score

In [6]:
from evaluate import load
bertscore = load("bertscore")
result = bertscore.compute(predictions=predict_KPs, references=target_KPs, lang="en")['f1']
print(result)
print(sum(result)/len(result))

[0.8635597825050354, 0.8843043446540833, 0.9038675427436829, 0.8991906046867371, 0.9360724687576294, 0.8928770422935486, 0.9285079836845398, 0.9046474099159241, 0.9327081441879272, 0.9054399132728577, 0.8559128046035767, 0.8982465863227844, 0.8887012004852295, 0.8725323677062988, 0.9263206720352173, 0.8740799427032471, 0.8998826742172241, 0.903356671333313, 0.9069432020187378, 0.9018583297729492, 0.9086926579475403, 0.9104728698730469, 0.8922751545906067, 0.9118797183036804, 0.8687792420387268, 0.9241908192634583, 0.9179991483688354, 0.9140012860298157, 0.9445348381996155, 0.9116018414497375, 0.9173768758773804, 0.9014414548873901, 0.9322829842567444, 0.9099816679954529, 0.936376690864563, 0.9139307737350464, 0.9122003316879272, 0.9266487956047058, 0.9250168800354004, 0.888757586479187, 0.9434616565704346, 0.9298383593559265, 0.9359979629516602, 0.9351403117179871, 0.9216471910476685, 0.9478394389152527, 0.9102783799171448, 0.948121964931488, 0.9301407337188721, 0.8944240808486938, 0.8

### Visualize result in txt

In [8]:
f = open("./prompt/gpt-4+one shot.txt",'w', encoding='UTF-8')
for batch_id, kp in enumerate(target_KPs):
    print(batch_id)
    print("-------------------Processing-------------------")
    f.write(str(batch_id)+"\n")
    f.write("argument:\n")
    f.write(arguments[batch_id]+"\n")
    f.write("target:\n")
    f.write(target_KPs[batch_id]+"\n")
    f.write("predict:\n")
    f.write(predict_KPs[batch_id]+"\n")
    f.write("\n")

print("-----------------Finished--------------------------")

0
-------------------Processing-------------------
1
-------------------Processing-------------------
2
-------------------Processing-------------------
3
-------------------Processing-------------------
4
-------------------Processing-------------------
5
-------------------Processing-------------------
6
-------------------Processing-------------------
7
-------------------Processing-------------------
8
-------------------Processing-------------------
9
-------------------Processing-------------------
10
-------------------Processing-------------------
11
-------------------Processing-------------------
12
-------------------Processing-------------------
13
-------------------Processing-------------------
14
-------------------Processing-------------------
15
-------------------Processing-------------------
16
-------------------Processing-------------------
17
-------------------Processing-------------------
18
-------------------Processing-------------------
19
-------------------

### 3 Candidates+CoT

In [14]:
predict_KPs = []
system_prompt = \
        """
        You need to do key point analysis on the query of user and
        generate a key point from it. You need to do it step by step. Firstly, generate 3 candidates
        from the given query and then, select a best one from them and it should be in this format:
        Key Point: ...
        """
for argument in arguments:
    rep = openai.ChatCompletion.create(
        model = model_name,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": argument}
        ]
    )

    result = rep["choices"][0]["message"]["content"]
    print(len(predict_KPs))
    print(result)
    predict_KPs.append(result)

0
Step 1: Generate 3 Candidate Key Points:
1. The palatable potential of discovering a cure or treatment shortly after a life has been ended unnecessarily.
2. The ethical implications of ending a life prematurely when a possible cure or treatment might be found in the near future.
3. The agony of realizing a cure or treatment was found shortly after having ended someone's life unnecessarily.

Step 2: Select the Best Key Point:
Key Point: The ethical implications of ending a life prematurely when a possible cure or treatment might be found in the near future.
1
Step 1 - Generate 3 Candidates:
1. Importance of patient autonomy in deciding level of care.
2. Concept of "enough care" from the patient's perspective. 
3. Patient's right to decide the end of their care.

Step 2 - Select the Best One:

Key Point: Patient's active involvement and decision making in their healthcare process.
2
Step 1: Generate 3 Candidates

Candidate 1: Emphasizes on the personal right of an individual to end the

In [15]:
print(predict_KPs)

["Step 1: Generate 3 Candidate Key Points:\n1. The palatable potential of discovering a cure or treatment shortly after a life has been ended unnecessarily.\n2. The ethical implications of ending a life prematurely when a possible cure or treatment might be found in the near future.\n3. The agony of realizing a cure or treatment was found shortly after having ended someone's life unnecessarily.\n\nStep 2: Select the Best Key Point:\nKey Point: The ethical implications of ending a life prematurely when a possible cure or treatment might be found in the near future.", 'Step 1 - Generate 3 Candidates:\n1. Importance of patient autonomy in deciding level of care.\n2. Concept of "enough care" from the patient\'s perspective. \n3. Patient\'s right to decide the end of their care.\n\nStep 2 - Select the Best One:\n\nKey Point: Patient\'s active involvement and decision making in their healthcare process.', "Step 1: Generate 3 Candidates\n\nCandidate 1: Emphasizes on the personal right of an i

In [16]:
post_processing_predict = []
for a in predict_KPs:
    key_point_index = a.find("Key Point: ")
    print(key_point_index)
    key_point_sentence = a[key_point_index + len("Key Point:"):].strip()
    print(key_point_sentence)
    post_processing_predict.append(key_point_sentence)

429
The ethical implications of ending a life prematurely when a possible cure or treatment might be found in the near future.
238
Patient's active involvement and decision making in their healthcare process.
378
An individual has the right to end their own suffering and any compassionate person assisting them should not be legally penalized.
254
The right of a person to determine how they die.
348
Recognition of individual's right to choose between life and death.
299
The right to dictate the terms of one's own death.
293
Forcing a person to suffer when death is inevitable seems inhumane.
334
The potential misuse of assisted suicide by greedy relatives for financial benefit.
304
Assisted suicide, even when consented to, is regarded as murder as it involves the act of deliberately causing someone's death.
343
Ethical considerations in medical decisions: Recovery through alternative treatments versus choosing to end life, especially when the decision involves others.
276
Legal consequen

### BLEU Score

In [19]:
import nltk
print("BLEU Score:  "+ str(nltk.translate.bleu_score.corpus_bleu(target_KPs, post_processing_predict)))

BLEU Score:  1.1416230948155883e-231


D:\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [17]:
import sacrebleu

# Example sentences
candidates = post_processing_predict
references = target_KPs

# Join sentences into strings
candidate_str = "\n".join(candidates)
reference_str = "\n".join(references)

# Calculate BLEU score
bleu = sacrebleu.raw_corpus_bleu(candidate_str, [reference_str])
print("BLEU Score:", bleu.score)

BLEU Score: 5.9417566932832315


### BERT Score

In [18]:
from evaluate import load
bertscore = load("bertscore")
result = bertscore.compute(predictions=post_processing_predict, references=target_KPs, lang="en")['f1']
print(result)
print(sum(result)/len(result))

[0.8494605422019958, 0.8639979362487793, 0.8519822955131531, 0.893051266670227, 0.8799841403961182, 0.8867245316505432, 0.8509701490402222, 0.8823188543319702, 0.9054784774780273, 0.8507179021835327, 0.8579479455947876, 0.875542402267456, 0.8940643072128296, 0.8684825301170349, 0.9206469655036926, 0.8672282695770264, 0.8856934905052185, 0.9153993725776672, 0.8776513934135437, 0.9009321928024292, 0.9056463837623596, 0.848436713218689, 0.8785206079483032, 0.9033403992652893, 0.8662371635437012, 0.9262098073959351, 0.8914957046508789, 0.9121692180633545, 0.9128230214118958, 0.9013446569442749, 0.9253867268562317, 0.9121492505073547, 0.8761538863182068, 0.8746325373649597, 0.931999921798706, 0.9166098833084106, 0.8584637641906738, 0.860936164855957, 0.9275503158569336, 0.879912793636322, 0.9479963183403015, 0.9232432842254639, 0.8905869722366333, 0.8928776383399963, 0.8687419891357422, 0.9580085277557373, 0.8827306032180786, 0.8819767832756042, 0.8824608325958252, 0.8723357319831848, 0.903

In [19]:
f = open("./prompt/gpt-4+CoT+3Candidates.txt",'w', encoding='UTF-8')
for batch_id, kp in enumerate(target_KPs):
    print(batch_id)
    print("-------------------Processing-------------------")
    f.write(str(batch_id)+"\n")
    f.write("argument:\n")
    f.write(arguments[batch_id]+"\n")
    f.write("target:\n")
    f.write(target_KPs[batch_id]+"\n")
    f.write("predict:\n")
    f.write(predict_KPs[batch_id]+"\n")
    f.write("\n")

print("-----------------Finished--------------------------")

0
-------------------Processing-------------------
1
-------------------Processing-------------------
2
-------------------Processing-------------------
3
-------------------Processing-------------------
4
-------------------Processing-------------------
5
-------------------Processing-------------------
6
-------------------Processing-------------------
7
-------------------Processing-------------------
8
-------------------Processing-------------------
9
-------------------Processing-------------------
10
-------------------Processing-------------------
11
-------------------Processing-------------------
12
-------------------Processing-------------------
13
-------------------Processing-------------------
14
-------------------Processing-------------------
15
-------------------Processing-------------------
16
-------------------Processing-------------------
17
-------------------Processing-------------------
18
-------------------Processing-------------------
19
-------------------

### Chain of thought

In [24]:
predict_KPs = []
system_prompt = \
        """
        You need to do key point analysis on the query of user and
        generate a key point from it. Let's do it step by step. Return the whole precess.
        And the key point should be in this format: Key Point: ...
        """
for argument in arguments:
    rep = openai.ChatCompletion.create(
        model = model_name,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": argument}
        ]
    )

    result = rep["choices"][0]["message"]["content"]
    print(len(predict_KPs))
    print(result)
    predict_KPs.append(result)

0
Step 1: Understanding the Query 
The user implies a scenario where someone's life ended perhaps due to a disease or condition and soon after, a cure or treatment was found. They intend to stress the tragedy of such timing.

Step 2: Identifying the Key Elements 
1. The discovery of a cure or treatment.
2. The ending of someone's life.

Step 3: Analyzing the Relation Between the Key Elements 
The user presents a causal relationship between the key elements; the discovery of a treatment comes after the life-ending event, implying a regrettable and tragic circumstance.

Step 4: Formulating the Key Point
Following these steps, we can then come up with a key point from this query.

Key Point: The tragic scenario of discovering a treatment shortly after a patient's life has ended prematurely.
1
Step 1: Understand the context - Here user is talking about a patient's right to decide about their own healthcare.

Step 2: Identify the main elements - Patient, decide, enough care.

Step 3: Break 

In [26]:
post_processing_predict = []
for a in predict_KPs:
    key_point_index = a.find("Key Point: ")
    print(len(post_processing_predict))
    key_point_sentence = a[key_point_index + len("Key Point:"):].strip()
    print(key_point_sentence)
    post_processing_predict.append(key_point_sentence)

0
The tragic scenario of discovering a treatment shortly after a patient's life has ended prematurely.
1
Autonomy of patient in deciding when their healthcare treatment is sufficient.
2
The right to personal autonomy allows an individual in pain to choose to end their suffering, and the person who aids in this process should not face punishment.
3
The dignity and autonomy of a person's decision concerning their manner of death should be respected.
4
Advocacy for an individual's right to decide whether they want to live or die.
5
A person should have the autonomy to make decisions about their own life-ending procedures.
6
The user believes that individuals should not be compelled to endure suffering, particularly when their demise is unavoidable.
7
The potential risk of exploitation of vulnerable individuals by opportunistic relatives in the case of legalizing assisted suicide for financial benefits.
8
Assisted suicide should be considered as murder regardless of the person's consent.
9

In [29]:
post_processing_predict[19]="Assisted suicide provides terminally ill individuals with the choice to control their death, mitigating prolonged pain and fostering dignity."
post_processing_predict[23]="Assisted suicide provides people the autonomy to control their life in dire situations and offers dignity in death."
post_processing_predict[30]="Assisted suicide is considered a criminal act which should result in imprisonment for those involved."
post_processing_predict[39]="Assisted suicide, being an individual's personal choice pertaining to their life, should be constitutionally legal."
post_processing_predict[42]="User equates assisted suicide with murder."
post_processing_predict[67]="Assisted suicide should be deemed a criminal offence as it leads to the death of a human being, equating it with criminal acts that cause loss of life."
post_processing_predict[94]="Assisted suicide should be decriminalized as it can provide a peaceful end for those suffering from debilitating medical conditions such as motor neurone disease."
post_processing_predict[95]="Assisted suicide should be decriminalized to provide terminally ill patients with the option to die with dignity rather than endure prolonged suffering."

Assisted suicide provides terminally ill individuals with the choice to control their death, mitigating prolonged pain and fostering dignity.


### BLEU Score

In [30]:
import sacrebleu

# Example sentences
candidates = post_processing_predict
references = target_KPs

# Join sentences into strings
candidate_str = "\n".join(candidates)
reference_str = "\n".join(references)

# Calculate BLEU score
bleu = sacrebleu.raw_corpus_bleu(candidate_str, [reference_str])
print("BLEU Score:", bleu.score)

BLEU Score: 5.130618969169216


### BERT Score

In [31]:
from evaluate import load
bertscore = load("bertscore")
result = bertscore.compute(predictions=post_processing_predict, references=target_KPs, lang="en")['f1']
print(result)
print(sum(result)/len(result))

[0.8479183316230774, 0.8609330654144287, 0.8517807722091675, 0.8991422057151794, 0.8905936479568481, 0.9319244027137756, 0.8481616973876953, 0.8780943155288696, 0.9219678044319153, 0.8375684022903442, 0.8482745885848999, 0.8632510304450989, 0.8856818675994873, 0.8589779734611511, 0.9210368394851685, 0.8656473755836487, 0.9028059244155884, 0.8719504475593567, 0.9087396860122681, 0.9017077684402466, 0.901710033416748, 0.8912273645401001, 0.8807159662246704, 0.8734064102172852, 0.8575495481491089, 0.9246174097061157, 0.923613429069519, 0.9210597276687622, 0.9146339297294617, 0.904748797416687, 0.8364540338516235, 0.9071860313415527, 0.9114923477172852, 0.9251725673675537, 0.886370062828064, 0.9129204750061035, 0.909584105014801, 0.9006735682487488, 0.8863122463226318, 0.8962922692298889, 0.9307327270507812, 0.9509995579719543, 0.8761151432991028, 0.9285438060760498, 0.8764835000038147, 0.9017339944839478, 0.88206946849823, 0.8905175924301147, 0.8856633901596069, 0.8764712810516357, 0.9188

In [33]:
f = open("./prompt/gpt-4+CoT.txt",'w', encoding='UTF-8')
for batch_id, kp in enumerate(target_KPs):
    print(batch_id)
    print("-------------------Processing-------------------")
    f.write(str(batch_id)+"\n")
    f.write("argument:\n")
    f.write(arguments[batch_id]+"\n")
    f.write("target:\n")
    f.write(target_KPs[batch_id]+"\n")
    f.write("predict:\n")
    f.write(predict_KPs[batch_id]+"\n")
    f.write("\n")

print("-----------------Finished--------------------------")

0
-------------------Processing-------------------
1
-------------------Processing-------------------
2
-------------------Processing-------------------
3
-------------------Processing-------------------
4
-------------------Processing-------------------
5
-------------------Processing-------------------
6
-------------------Processing-------------------
7
-------------------Processing-------------------
8
-------------------Processing-------------------
9
-------------------Processing-------------------
10
-------------------Processing-------------------
11
-------------------Processing-------------------
12
-------------------Processing-------------------
13
-------------------Processing-------------------
14
-------------------Processing-------------------
15
-------------------Processing-------------------
16
-------------------Processing-------------------
17
-------------------Processing-------------------
18
-------------------Processing-------------------
19
-------------------